In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer,RobertaTokenizer
# import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch
from langchain.chains.llm import LLMChain
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import warnings
import time
from langchain.chains import SimpleSequentialChain
from langchain.prompts.chat import ChatPromptTemplate
warnings.filterwarnings("ignore")

Objective : Information retrieval using Question Answering model about crimes commited and when was the crimes committed.

Conclusion : ConversationalRetrievalChain was performing best,as it was able to extract correct answers for most of questions.

Implementation & approach : 
Applying a for loop for each report in biography section
1)Documents creation
2)Splitting it into chunks - Now used tokentextsplitter to create chunks because the chunk size should not exceed token limit of llm model.Kept chunk size 300 with overalp of 50 tokens.So mostly the complete document was created as one chunk.(Normally the approach for each chunk they should contain specific unique information.So we can use semantic chunks or some manual obersvation that after 4-5 lines specify one particular information, hence chunk size should decide based on that).
3)Need to create embeddings for chunks and store them using vector database. So use OllamaEmbeddings and Store in FAISS
4)Now we need to retrieve only relevant chunks based on query we pass, so there are many retrievers to compare against likve vector store , contextualcompression , multi query etc. Need to check there out put cunks based on query we provide.In our case since there was only one chunk being prepared every time hence there is no impact on retrievers .We used vector store as retriever 
5)Now we have 3 qa chains to compare performance - Retrieval QA Chain , Conversationa Retrieval Chain , Load qa chain. Out of which Conversationa Retrieval Chain was getting good answers among all.



6)Used prompt template to get defined answers to two questions asked.For instance, model should predict answers in few words and if doesn't get answers it should provide cobtext in few line





In [2]:
import os
#doc Loaders
import langchain
from langchain.llms import OpenAI
from langchain.document_loaders import CSVLoader,HuggingFaceDatasetLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import Chroma,FAISS

from langchain_community.chat_models import ChatOllama
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.text_splitter import TokenTextSplitter

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [41]:
# text_splitter=SemanticChunker(OllamaEmbeddings())
# docs = text_splitter.create_documents()
# print(docs[0].page_content)

In [4]:
import pandas as pd
pd.set_option("display.max_colwidth",None)

### Document Loaders

In [5]:
from langchain.document_loaders import CSVLoader
import csv

class CustomCSVLoader(CSVLoader):
    def __init__(self, file_path, column_name):
        super().__init__(file_path)
        self.column_name = column_name

    def load(self):
        documents = []
        with open(self.file_path, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if self.column_name in row:
                    # Only keep the specified column
                    documents.append({self.column_name: row[self.column_name]})
        return documents

# Usage
file_path = 'Bio_with_only_reports_csv_1.csv'
# column_name = 'V_BIOGRAPHY_repo'

loader = CSVLoader(file_path=file_path)
documents = loader.load()

In [6]:
df_repo=pd.read_csv("Bio_with_only_reports_csv_1.csv")

In [7]:
df_repo["V_BIOGRAPHY_repo"].loc[1]

' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'

In [46]:
# documents

### Chunking - Character , Recursive Character , Semantic 

In [8]:
chunk_size =300
chunk_overlap = 50

# RecursiveCaracterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)




# TikTokenencoder
r_tok_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)

# text_splitter = TokenTextSplitter(chunk_size=300, chunk_overlap=50)

In [48]:
# db = Chroma.from_documents(docs, embedding=embeddings,persist_directory='C:/Users/MBBLABS')
# db = FAISS.from_documents(docs, embedding=embeddings)

# db.persist()

# VectorStore Retriever

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOllama

In [11]:
chunk_size =300
chunk_overlap = 50

# TikTokenencoder
r_tok_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)

In [12]:
embedding=OllamaEmbeddings(model="llama2:7b")
# db = Chroma.from_documents(docs, embedding=embeddings,persist_directory='C:/Users/MBBLABS')
# db = FAISS.from_documents(docs, embedding=embeddings)

# # db.persist()

In [13]:
# question = "What is the crime committed in few words?"
prompt_template = """Answer the question in few words using the provided context in keywords only. If the answer is
                    not contained in the context, then return context in one line. \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [14]:
df_repo["Crime_predicted"]=""
df_repo["time_for_crime"]=0
df_repo["Date_of_crime_predicted"]=""
df_repo["time_for_date_of_crime"]=0

### Langchain LLM only with prompt

In [55]:
# for i in range(10):
#     docs=r_tok_splitter.split_documents(documents[i:i+1])   # Tiktoken RCT 
#     db=FAISS.from_documents(docs,embedding=embedding)       # Embeddings stor into Vectorstore
#     retriver=db.as_retriever()                              # Vector store retriever
#     conversational_chain = ConversationalRetrievalChain.from_llm(llm=ChatOllama(model="llama2:7b"),retriever=db.as_retriever(),combine_docs_chain_kwargs={"prompt": prompt})

#     # query 1
#     print("Bio "+str(i)+":",df_repo["V_BIOGRAPHY_repo"].loc[i],end="\n\n")
#     query = 'The individual is involved in which crime in few words?'
#     chat_history = []
#     str1=time.time()
#     result = conversational_chain.run(chat_history=chat_history, question=query)
#     end1=time.time()
#     print(query,end="\n\n")
#     chat_history.append((query, result))
#     df_repo["Crime_predicted"].loc[i]=result
#     df_repo["time_for_crime"].loc[i]=end1-str1
#     print("Answer 1: " + result+"\n\n")
#     # query 2
#     query = 'When was it committed in few words ?'
#     print(query,end="\n\n")
#     str2=time.time()
#     result = conversational_chain.run(chat_history=chat_history, question=query)
#     end2=time.time()
#     df_repo["time_for_date_of_crime"].loc[i]=end2-str2
#     df_repo["Date_of_crime_predicted"].loc[i]=result
#     print("Answer 2: " + result+"\n\n")
#     chat_history.append((query, result))

### Langchains

In [ ]:
for i in range(10):
    docs=r_tok_splitter.split_documents(documents[i:i+1])   # Tiktoken RCT 
    db=FAISS.from_documents(docs,embedding=embedding)       # Embeddings stor into Vectorstore
    retriver=db.as_retriever()                              # Vector store retriever
    conversational_chain = ConversationalRetrievalChain.from_llm(llm=ChatOllama(model="llama2:7b"),retriever=db.as_retriever(),combine_docs_chain_kwargs={"prompt": prompt})

    # query 1
    print("Bio "+str(i)+":",df_repo["V_BIOGRAPHY_repo"].loc[i],end="\n\n")
    query = 'The individual is involved in which crime in few words?'
    chat_history = []
    str1=time.time()
    result = conversational_chain.run(chat_history=chat_history, question=query)
    end1=time.time()
    print(query,end="\n\n")
    chat_history.append((query, result))
    df_repo["Crime_predicted"].loc[i]=result
    df_repo["time_for_crime"].loc[i]=end1-str1
    print("Answer 1: " + result+"\n\n")
    
    # query 2
    query = 'When was it committed in few words ?'
    print(query,end="\n\n")
    str2=time.time()
    result = conversational_chain.run(chat_history=chat_history, question=query)
    end2=time.time()
    df_repo["time_for_date_of_crime"].loc[i]=end2-str2
    df_repo["Date_of_crime_predicted"].loc[i]=result
    print("Answer 2: " + result+"\n\n")
    chat_history.append((query, result))

In [57]:
chat_history

[('The individual is involved in which crime in few words?',
  'The individual is involved in "dacoity" according to the context.'),
 ('When was it committed in few words ?',
  'The answer to the question "When did the dacoity occur?" can be found in the context as follows:\n\n"2009"')]

In [58]:
df_repo.loc[:100]

,V_BIOGRAPHY_repo,Crime_predicted,time_for_crime,Date_of_crime_predicted,time_for_date_of_crime
0,"mar 2022 - no further information reported. [note] this record supports the inclusion criteria for individuals connected, associated, affiliated, linked to, or who own or control a sanctioned entity (ofac sanctions list).",The individual is involved in no crime.,20.531514,Incident occurred in March 2022.,33.993145
1,"dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",Bribery,31.016457,Bribery occurred in July 2019 - October 2019.,46.852337
2,"dec 2018 - reportedly detained. sentenced to death for killing law enforcement agents and destruction of government educational institutions. apr 2022 - no further information reported.[crime - terror category notice] this category includes information about individuals who have reportedly been convicted of a terror related crime. inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.","\nThe individual is involved in terror-related crimes in few words: ""killing law enforcement agents and destruction of government educational institutions.""",38.854634,"The act of terrorism took place in December 2018, as reported in the context.",54.945599
3,jun 2018 - declared as proclaimed offender by khairpur court for involvement in organised crime related activities. may 2022 - no further information reported.,Involved in organized crime.,20.819613,"\nThe answer to the question ""When did the organized crime take place?"" can be found in the context as follows:\n\n""jun 2018""",36.982092
4,"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, arms and ammunition. jun 2022 - no further information reported.",\nTheft,20.867172,Theft occurred in Aug 2013.,39.722976
...,...,...,...,...,...
96,jun 2011 - sentenced to an unspecified term of imprisonment for illegal on line gambling. oct 2016 - no further information reported.,,0.000000,,0.000000
97,"mar 2015 - left for syria. 2017 - arrested by syrian democratic forces. nov 2019 - deported from syria. detained under malaysia's security offenses (special measures) act or sosma upon arriving in kuala lumpur. dec 2019 - charged with travelling to a foreign country to commit terrorist acts. jul 2020 - pleaded not guilty to charges. oct 2020 - trial postponed. apr 2022 - designated by the myagc-sanc for participating and facilitating in the commission of a terrorist act. oct 2022 - no further information reported.[nonconviction terror category notice] this category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. the category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. individuals included in this category ha

In [59]:
df_repo["Crime_extracted"]=df_repo["Crime_predicted"].apply(lambda x: "" if pd.isna(x) else x)
df_repo["Crime_predicted"]=df_repo["Crime_extracted"].apply(lambda x: re.sub("\n","",x))
df_repo["Date_of_crime_extracted"]=df_repo["Date_of_crime_predicted"].apply(lambda x: "" if pd.isna(x) else x)
df_repo["Date_of_crime_extracted"]=df_repo["Date_of_crime_extracted"].apply(lambda x: re.sub("\n","",x))

In [60]:
%%time
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(temperature=0, model = "llama3")

CPU times: total: 0 ns
Wall time: 0 ns


In [61]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


df_repo["crime_entity"]=""
df_repo["date_entity"]=""
system_prompt = """
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and date if it is present. else dont provide any information. 
 provide in json format having keys as crime and date.If not able to extract both entities, then provide json with same keys but empty string as values.
 Please do not provide any additional text apart from json"""
crime_schema = ResponseSchema(name="crime",
                                   description="This refers to crime entity present in given text.")

date_schema = ResponseSchema(name="date",
                                       description="This is refers to date present in given text.")


response_schema = [
    crime_schema ,
    date_schema,
]

# prompt_template = ChatPromptTemplate.from_template(template)
# messages = prompt_template.format_messages(text=text, format_instructions=format_instructions)

output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)
df_repo["time_of_entity_extr"]=0

df_repo["Crime_date_combined"]=df_repo["Crime_extracted"]+" "+df_repo["Date_of_crime_extracted"]
texts=df_repo["Crime_date_combined"].loc[:10]

for i,text in zip(range(len(texts)),texts):
    print(text)
# text = """ Jun 2021 - reportedly sentenced to 10 years imprisonment by Muse District Court for human trafficking. Mar 2023 - no further information reported."""
# df.loc[:1,"Crime_extracted"]
    prompt_template_entity = ChatPromptTemplate.from_messages([ ("system", f'''{system_prompt}'''), ("human", "{user_input}"),])
    prompt_template_entity.input_variables
    start=time.time()
    # final_prompt=prompt_template_entity.format_prompt(user_input=text).to_messages()
    # output=llm(final_prompt)
    llm_chain = LLMChain(llm=llm, prompt=prompt_template_entity, verbose=True)
    output = llm_chain.run(user_input= text)
    end=time.time()
    
    df_repo["time_of_entity_extr"]=end-start
    print(output)
    output_dict=  output_parser.parse(output)
    df_repo["crime_entity"].loc[i]=output_dict["crime"]
    df_repo["date_entity"].loc[i]=output_dict["date"]

    # print(output)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"crime": string  // This refers to crime entity present in given text.
	"date": string  // This is refers to date present in given text.
}
```
The individual is involved in no crime. Incident occurred in March 2022.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and date if it is present. else dont provide any information. 
 provide in json format having keys as crime and date.If not able to extract both entities, then provide json with same keys but empty string as values.
 Please do not provide any additional text apart from json
Human: The individual is involved in no crime. Incident occurred in March 2022.

> Finished chai

In [ ]:
# df_repo.to_csv("Crimes_&_date_predicted_new.csv")

In [62]:
df_repo

,V_BIOGRAPHY_repo,Crime_predicted,time_for_crime,Date_of_crime_predicted,time_for_date_of_crime,Crime_extracted,Date_of_crime_extracted,crime_entity,date_entity,time_of_entity_extr,Crime_date_combined
0,"mar 2022 - no further information reported. [note] this record supports the inclusion criteria for individuals connected, associated, affiliated, linked to, or who own or control a sanctioned entity (ofac sanctions list).",The individual is involved in no crime.,20.531514,Incident occurred in March 2022.,33.993145,The individual is involved in no crime.,Incident occurred in March 2022.,,March 2022,12.286842,The individual is involved in no crime. Incident occurred in March 2022.
1,"dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",Bribery,31.016457,Bribery occurred in July 2019 - October 2019.,46.852337,Bribery,Bribery occurred in July 2019 - October 2019.,Bribery,July 2019 - October 2019,12.286842,Bribery Bribery occurred in July 2019 - October 2019.
2,"dec 2018 - reportedly detained. sentenced to death for killing law enforcement agents and destruction of government educational institutions. apr 2022 - no further information reported.[crime - terror category notice] this category includes information about individuals who have reportedly been convicted of a terror related crime. inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.","The individual is involved in terror-related crimes in few words: ""killing law enforcement agents and destruction of government educational institutions.""",38.854634,"The act of terrorism took place in December 2018, as reported in the context.",54.945599,"\nThe individual is involved in terror-related crimes in few words: ""killing law enforcement agents and destruction of government educational institutions.""","The act of terrorism took place in December 2018, as reported in the context.",terrorism,December 2018,12.286842,"\nThe individual is involved in terror-related crimes in few words: ""killing law enforcement agents and destruction of government educational institutions."" The act of terrorism took place in December 2018, as reported in the context."
3,jun 2018 - declared as proclaimed offender by khairpur court for involvement in organised crime related activities. may 2022 - no further information reported.,Involved in organized crime.,20.819613,"\nThe answer to the question ""When did the organized crime take place?"" can be found in the context as follows:\n\n""jun 2018""",36.982092,Involved in organized crime.,"The answer to the question ""When did the organized crime take place?"" can be found in the context as follows:""jun 2018""",organized crime,jun 2018,12.286842,"Involved in organized crime. The answer to the question ""When did the organized crime take place?"" can be found in the context as follows:""jun 2018"""
4,"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, arms and ammunition. jun 2022 - no further information reported.",Theft,20.867172,Theft occurred in Aug 2013.,39.722976,\nTheft,Theft occurred in Aug 2013.,Theft,Aug 2013,12.286842,\nTheft Theft occurred in Aug 2013.
...,...,...,...,...,...,...,...,...,...,...,...
5744,"feb 2011 - fined inr200,000 by sebi for fr